# ODA data by Donor
This tutorial shows you how to build a Pandas DataFrame containing ODA data from any number of donors over a range of years and multiple different indicators.

The tutorial has the following steps:
1. Import `ODAData`, the main tool used to interact with the data
2. Set the arguments
3. Create an instance of `ODAData` with the specific arguments you would like to use
4. Load the indicators and get the DataFrame
5. Optionally, export the DataFrame

## 1. Import ODAData

We can gather the data we need using the `ODAData` class. An object from this class can:
- Get data for specific indicators
- Optionally, filter the data for specific donors, recipients and years
- Optionally, exchange and deflate data

`oda_data` automatically downloads the relevant datasets (i.e. DAC1, DAC2a or CRS tables) depending on the indicators selected in step 3.
It is highly recommended that you specify the folder where you want to store, and from where you want to read, this raw data. This must be done once in each notebook or script where we use the `oda_data` package. You can specify the data path if the raw data has already been downloaded, or if you haven't yet specified the download path in your notebook and script.

In [1]:
from oda_data import ODAData, set_data_path

# If you haven't set the data path, you can do it now.
# If you haven't downloaded the raw data before, please see step 0
set_data_path(path="../uk_oda_test/data")

# 2. Set the arguments for the DataFrame

Next, we need to set the right arguments, which are used to create the right DataFrame.
- *years*: you must specify the `years`, as an `int`, `list` or `range`
- *donors*: you can _optionally_ specify the `donors` you want the output to have (as `int`, or `list`) of donor codes.
- *recipients*: you can _optionally_ specify the `recipients` you want the output to have. Not all indicators
    need or accept recipients. If using an indicator for which recipients are not an option, a warning will be logged to
    the console and the recipients ignored for that indicator.
- *currency*: you can _optionally_ specify the `currency` in which you want your data to be shown. If not specified,
    by default, `USD` will be used. Other options include `EUR`, `GBP` and `CAN`.
- *prices*: you can _optionally_ specify the `prices` in which you want your data to be shown. If not specified,
    by default, `current` will be used. The other option is `constant`. If specifying `constant` a `base_year` must be set.
- *base_year*: you must specify a `base_year` if you have set `prices = 'constant'`. If you have chosen `current` prices,
    by default, `base_year` will be `None`.


Below are some example settings for this tutorial. For clarity, we will first store them in variables, but you can
always pass them directly as arguments to the `ODAData` class.

In [1]:
# Select years as (for example) a range. Remember ranges are exclusive of the upper bound.
years = range(2012, 2022)

# Select donors, which must be specified by their codes. Here, we are selecting the UK only.
donors = [12]

# Select the currency. By default 'USD' is shown but we'll get the data in British Pounds.
currency = 'GBP'

# Select the prices. By default, 'current' is shown, but we'll get the data in constant prices.
prices = 'constant'

# Set the base year. We must set this given that we've asked for constant data.
base_year = 2021

To see a list of available indicators, donor codes, and currencies, you can use a few methods provided by the ODAData object.

For example:

In [3]:
ODAData().available_donors()

INFO 2023-01-17 16:29:20,131 [oda_data.py:available_donors:445] Note that not all donors may be available for all indicators


{
1: Austria,
2: Belgium,
3: Denmark,
4: France,
5: Germany,
6: Italy,
7: Netherlands,
8: Norway,
9: Portugal,
10: Sweden,
11: Switzerland,
12: United Kingdom,
18: Finland,
20: Iceland,
21: Ireland,
22: Luxembourg,
40: Greece,
50: Spain,
61: Slovenia,
68: Czech Republic,
69: Slovak Republic,
75: Hungary,
76: Poland,
301: Canada,
302: United States,
701: Japan,
742: Korea,
801: Australia,
820: New Zealand,
30: Cyprus,
45: Malta,
55: Turkey,
62: Croatia,
70: Liechtenstein,
72: Bulgaria,
77: Romania,
82: Estonia,
83: Latvia,
84: Lithuania,
87: Russia,
130: Algeria,
133: Libya,
358: Mexico,
543: Iraq,
546: Israel,
552: Kuwait,
561: Qatar,
566: Saudi Arabia,
576: United Arab Emirates,
611: Azerbaijan,
613: Kazakhstan,
732: Chinese Taipei,
764: Thailand,
765: Timor-Leste,
104: Nordic Development Fund,
807: UNEP,
811: Global Environment Facility,
812: Montreal Protocol,
901: International Bank for Reconstruction and Development,
902: Multilateral Investment Guarantee Agency,
903: Internationa

In [4]:
ODAData().available_currencies()

[
USD,
EUR,
GBP,
CAD
]

Finally, we can specify the indicators we want to retrieve for our analysis.
A full list of indicators can be seen by using the `.available_indicators()` method.

In [5]:
ODAData().available_indicators()

[
total_oda_flow_net,
total_oda_ge,
total_oda_official_definition,
total_oda_bilateral_flow_net,
total_oda_bilateral_ge,
total_oda_multilateral_flow_net,
total_oda_multilateral_ge,
total_oda_flow_gross,
total_oda_flow_commitments,
total_oda_grants_flow,
total_oda_grants_ge,
total_oda_non_grants_flow,
total_oda_non_grants_ge,
total_covid_oda_ge,
total_covid_oda_flow,
total_covid_oda_ge_linked,
total_health_covid_oda_ge,
total_health_covid_oda_flow,
total_health_covid_oda_ge_linked,
total_covid_vaccine_donations_oda_ge,
total_covid_vaccine_donations_oda_flow,
total_covid_vaccine_donations_oda_ge_linked,
total_covid_vaccine_donations_domestic_supply_oda_ge,
total_covid_vaccine_donations_domestic_supply_oda_flow,
total_covid_vaccine_donations_domestic_supply_oda_ge_linked,
total_covid_vaccine_donations_dev_purchase_oda_ge,
total_covid_vaccine_donations_dev_purchase_oda_flow,
total_covid_vaccine_donations_dev_purchase_oda_ge_linked,
total_covid_ancillary_oda_ge,
total_covid_ancillary_oda_fl

We are interested in Total ODA, bilateral vs multilateral ODA, grants vs non-grants, idrc, and core ODA. Hence, we need to load the following indicators (plus a few extras for additional analysis).

In [6]:
indicators = ['total_oda_official_definition', 'total_oda_ge','total_oda_grants_ge','total_oda_non_grants_ge','gni','total_oda_bilateral_ge','total_oda_multilateral_ge','total_oda_flow_net','total_oda_grants_flow','total_oda_non_grants_flow','total_oda_non_grants_ge','total_oda_bilateral_flow_net','total_oda_multilateral_flow_net', 'idrc_flow', 'idrc_ge', 'idrc_ge_linked', 'one_core_oda_flow', 'one_core_oda_ge', 'one_core_oda_ge_linked','total_psi_ge', 'institutional_psi_ge', 'instrument_psi_ge']

## 3. Create an instance of ODAData

In order to get the data we want, we will create an instance of the `ODAData` class, using the arguments we specified above.

We will store this instance in a variable called `oda`, which we will use later to load the indicators and get the DataFrame we're after.


In [7]:
oda = ODAData(years=years,
              donors=donors,
              currency=currency,
              prices=prices,
              base_year=base_year,
              include_names=True)

## 4. Load the indicators and get the DataFrame

Then we loop through all your assigned indicators and tell ODAData to load them.

We have added the method `add_share_of_total` which calculates the relevant share of the indicator of the donors total.

In [8]:
# Add all the indicators in our `indicators` list
for indicator in indicators:
    oda.load_indicator(indicator)

# get a DataFrame with all the dat
df = oda.add_share_of_total(True).get_data('all')

# show the resulting dataframe
df

,year,indicator,donor_code,donor_name,share_of,currency,prices,value,share
0,2012,gni,12,United Kingdom,NaN,GBP,constant,1.876459e+06,NaN
1,2012,idrc_flow,12,United Kingdom,total_oda_flow_net,GBP,constant,3.412661e+01,0.32365
2,2012,idrc_ge_linked,12,United Kingdom,NaN,GBP,constant,3.412661e+01,NaN
3,2012,one_core_oda_flow,12,United Kingdom,total_oda_flow_net,GBP,constant,1.039835e+04,98.61670
4,2012,total_oda_bilateral_flow_net,12,United Kingdom,total_oda_flow_net,GBP,constant,6.577411e+03,62.37935
...,...,...,...,...,...,...,...,...,...
151,2021,total_oda_multilateral_ge,12,United Kingdom,total_oda_ge,GBP,constant,4.276919e+03,37.44250
152,2021,total_oda_non_grants_flow,12,United Kingdom,total_oda_flow_net,GBP,constant,8.975469e+02,7.58451
153,2021,total_oda_non_grants_ge,12,United Kingdom,total_oda_ge,GBP,constant,8.734614e+02,7.64676
154,2021,total_oda_official_definition,12,United Kingdom,total_oda_official_definition,GBP,constant,1.142263e+04,100.00000


## Optionally export DataFrame as CSV

Finally, we export the DataFrame as a CSV.

In [9]:
df.to_csv(r'../uk_oda_test/output/UK_oda_DAC1.csv', index = False)